In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import random
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor,ExtraTreesRegressor
from sklearn.model_selection import KFold
from catboost import CatBoostRegressor
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.neural_network import MLPRegressor
from catboost import CatBoost, Pool
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.preprocessing import RobustScaler,PowerTransformer,QuantileTransformer,Normalizer,MinMaxScaler
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-jan-2021/train.csv')
test  = pd.read_csv('../input/tabular-playground-series-jan-2021/test.csv')
sub = pd.read_csv('../input/tabular-playground-series-jan-2021/sample_submission.csv')
soumaya_sub = pd.read_csv('../input/soumaya-sub/submission.csv')

In [ ]:
columns = [col for col in train.columns.to_list() if col not in ['id','target']]

## Xgboost model

In [ ]:
param1={'tree_method':'gpu_hist','lambda': 0.0045523892919572, 'alpha': 0.005803668702291496,
            'colsample_bytree': 0.4, 'subsample': 0.8, 'learning_rate': 0.007, 'max_depth': 20,
            'random_state': 24, 'min_child_weight': 264,'n_estimators': 8200}

In [ ]:
preds1 = np.zeros(test.shape[0])
kf = KFold(n_splits=12,random_state=48,shuffle=True)
rmse=[]  ## list contains rmse for each fold
n=0
for trn_idx, test_idx in kf.split(train[columns],train['target']):
    X_tr,X_val=train[columns].iloc[trn_idx],train[columns].iloc[test_idx]
    y_tr,y_val=train['target'].iloc[trn_idx],train['target'].iloc[test_idx]
    model = xgb.XGBRegressor(**param1)
    model.fit(X_tr,y_tr,eval_set=[(X_val,y_val)],early_stopping_rounds=100,verbose=False)
    preds1+=model.predict(test[columns])/kf.n_splits
    rmse.append(mean_squared_error(y_val, model.predict(X_val), squared=False))
    print(n+1,rmse[n])
    n+=1  

### Xgb results : 
* CV = 0.69537
* LB result = 0.69712
* Private = 0.69608

## lgb model

In [ ]:
param2={'metric': 'rmse','lambda_l1': 6.177646935770319, 'lambda_l2': 8.267630169831212e-05, 
            'min_sum_hessian_in_leaf': 0.06121961712341018, 'min_data_in_leaf': 17, 'num_leaves': 256,'max_bin':456,
            'feature_fraction': 0.4808176760171774, 'bagging_fraction': 0.8799188937155665, 'learning_rate': 0.005, 
            'bagging_freq': 1, 'n_estimators': 15898, 'max_depth': 120, 'min_data_per_group': 53, 'random_state': 100, 'cat_smooth': 23}

In [ ]:
preds2 = np.zeros(test.shape[0])      
kf = KFold(n_splits=12,random_state=48,shuffle=True)   
rmse=[]     ## list contains rmse for each fold
n=0
for trn_idx, test_idx in kf.split(train[columns],train['target']): 
    X_tr,X_val=train[columns].iloc[trn_idx],train[columns].iloc[test_idx]
    y_tr,y_val=train['target'].iloc[trn_idx],train['target'].iloc[test_idx] 
    lgb = LGBMRegressor(**param2)
    lgb.fit(X_tr,y_tr,eval_set=[(X_val,y_val)],early_stopping_rounds=200,verbose=False)
    preds2+=lgb.predict(test[columns])/kf.n_splits
    rmse.append(mean_squared_error(y_val, lgb.predict(X_val), squared=False))
    print(n+1,rmse[n])    
    n+=1    

### Xgb results : 
* CV = 0.69494
* LB result = 0.69696
* Private = 0.69564

## Notes :
* I use ensemble of xgb and lgb (0.4xgb+0.6lgb) which gives me : LB result =  0.69686 and Private = 0.69565 !!! 🤷‍
* the LB score improoved 😀 but the private nope 😕
* I used also the output of [Somayyeh](https://www.kaggle.com/somayyehgholami) public kernel [LB result = 0.69652 and Private = 0.69529] and make ensemble with my work. [kernel of Somayyeh](https://www.kaggle.com/somayyehgholami/results-driven-tabular-playground-series-201).
* So my final submission was : 0.18 * lgb_output + 0.12 * xgb_output + 0.7 * soumayyeh_submission ==> 
LB result =  0.69648 and Private = 0.69525 🏄🏾‍♂️

##  This result ranked me 11/1728 in this competition 🏄🏽‍♂️
## I shouldn't forget to thank [Somayyeh](https://www.kaggle.com/somayyehgholami) 👏👏👏

# Submission

In [ ]:
sub=0.18*preds1+0.12*preds2+0.7*soumaya_sub['target']

In [ ]:
sub.to_csv('submission.csv', index=False)